In [ ]:

import os, sys, numpy as np, pandas as pd, torch
sys.path.append("..")
from src.config import ModelParams, TrainConfig
from src.deqn import PolicyNetwork, Trainer
from src.steady_states import solve_flexprice_sss, export_rbar_tensor, solve_taylor_sss
from src.io_utils import ensure_dir, save_torch, save_csv, save_json, pack_config, save_selected_run


In [ ]:
policy = "taylor"
params = ModelParams(device="cpu", dtype=torch.float32)

# --- reproducible run directory (unique; safe for parallel runs) ---
from src.io_utils import make_run_dir, save_run_metadata, pack_config
run_dir = make_run_dir("../artifacts", policy=policy, tag="cpu16gb_plan", seed=123)

# Strict training plan for a 16GB RAM CPU box:
#   Stage A: train with gh_n=3 (main training)
#   Stage B: short fine-tune with gh_n=5 (more accurate quadrature), with early stopping

cfgA = TrainConfig(
    seed=123,
    steps=20000,
    batch_size=256,          # keep as requested
    lr=1e-05,
    grad_clip=1.0,
    gh_n_train=3,
    replay_buffer_size=250_000,
    replay_refresh_every=120,
    replay_refresh_steps=80,
    replay_init_fill_steps=400,
    val_size=20000,
    val_every=500,
    reduce_lr_on_plateau=True,
    plateau_patience=3000,
    lr_reduce_factor=0.5,
    min_lr=2e-06,
    artifacts_root="../artifacts",
    run_dir=run_dir,
    save_best=True,
    enable_timers=True,
    log_every=50,
    profile=False,
    cpu_num_threads=None,
    cpu_num_interop_threads=None,
    matmul_precision="high",
)

cfgB = TrainConfig(
    seed=123,
    steps=2000,
    batch_size=64,           # gh_n=5 is expensive: keep batch small
    lr=3e-06,
    grad_clip=1.0,
    gh_n_train=5,
    replay_buffer_size=250_000,
    replay_refresh_every=200,
    replay_refresh_steps=60,
    replay_init_fill_steps=0,
    val_size=20000,
    val_every=250,
    early_stopping=True,
    patience=1500,
    min_delta=1e-5,
    artifacts_root="../artifacts",
    run_dir=run_dir,
    save_best=True,
    enable_timers=True,
    log_every=50,
    profile=False,
    cpu_num_threads=None,
    cpu_num_interop_threads=None,
    matmul_precision="high",
)


In [ ]:

flex = solve_flexprice_sss(params)
rbar = export_rbar_tensor(params, flex)


taylor_sss = solve_taylor_sss(params, flex)
print('Theoretical Taylor SSS:', taylor_sss.by_regime)


In [ ]:
import torch, numpy as np

def _quality_report(trainer, tag="", B_train=2048, B_val=4096, topk=5, commitment_sss=None, burn_in=200):
    """Post-training quality snapshot without relying on an internal replay buffer.

    We generate a fresh batch of states, optionally burn it in using the model's state transition,
    and then compute RMSE/max|res| for the residual system. For discretion/commitment residuals
    may require autograd (because we use autograd.grad inside), so we compute those under enable_grad.
    """
    dev, dt = trainer.params.device, trainer.params.dtype

    # --- build a state batch ---
    Xtr = trainer.simulate_initial_state(B_train, commitment_sss=commitment_sss if trainer.policy=="commitment" else None).to(dev, dt)

    # burn-in to get closer to ergodic region (no grad)
    if burn_in and burn_in > 0:
        with torch.inference_mode():
            x = Xtr
            for _ in range(int(burn_in)):
                x = trainer._step_state(x)
            Xtr = x

    # Validation batch: independent draw
    Xv = None
    if B_val and B_val > 0:
        Xv = trainer.simulate_initial_state(B_val, commitment_sss=commitment_sss if trainer.policy=="commitment" else None).to(dev, dt)
        if burn_in and burn_in > 0:
            with torch.inference_mode():
                x = Xv
                for _ in range(int(burn_in)):
                    x = trainer._step_state(x)
                Xv = x

    # choose grad context
    need_grad = trainer.policy in ("discretion", "commitment")

    def compute_stats(X):
        if X is None:
            return None
        ctx = torch.enable_grad() if need_grad else torch.inference_mode()
        with ctx:
            R = trainer._residuals(X)
            rmse_all = torch.sqrt((R**2).mean())
            rmse_eq = torch.sqrt((R**2).mean(dim=0))
            maxabs_eq = R.abs().max(dim=0).values
        # detach for printing
        return (rmse_all.detach().cpu().item(),
                rmse_eq.detach().cpu().numpy(),
                maxabs_eq.detach().cpu().numpy())

    tr = compute_stats(Xtr)
    va = compute_stats(Xv)

    labels = getattr(trainer, "res_keys", None)
    if labels is None:
        labels = [f"eq{i}" for i in range(len(tr[1]))]

    print(f"\n[quality snapshot] {trainer.policy} {tag}".strip())
    print(f"RMSE(all) train = {tr[0]:.4e}")
    if va is not None:
        print(f"RMSE(all) val   = {va[0]:.4e}")

    # rank by train RMSE (or val if present)
    ref = va[1] if va is not None else tr[1]
    worst_idx = np.argsort(ref)[::-1][:min(topk, len(ref))]

    print(f"Worst {min(topk,len(ref))} equations by RMSE:")
    for j in worst_idx:
        if va is not None:
            print(f"  {labels[j]:>18s}: train RMSE {tr[1][j]:.3e} | val RMSE {va[1][j]:.3e} | val max|res| {va[2][j]:.3e}")
        else:
            print(f"  {labels[j]:>18s}: train RMSE {tr[1][j]:.3e} | train max|res| {tr[2][j]:.3e}")


In [ ]:
from src.deqn import simulate_paths
x0 = trainer.simulate_initial_state(256)
sim = simulate_paths(params, policy, trainer.net, T=3000, burn_in=600, x0=x0, compute_implied_i=False)
for s in [0,1]:
    mask = (sim['s'] == s)
    print(f'Regime {s}: pi_sim={sim["pi"][mask].mean():.6f} | pi_sss={taylor_sss.by_regime[s]["pi"]:.6f}')


In [ ]:
# --- Save artifacts (final + pointers) ---
# We save final weights/logs to run_dir and also keep a pointer in selected_runs.json

# prefer Stage B config if it exists, else Stage A
_cfg_final = globals().get("cfgB", globals().get("cfgA"))
save_run_metadata(run_dir, pack_config(params, _cfg_final, extra={"policy": policy}))

save_torch(os.path.join(run_dir, "weights_final.pt"), trainer.net.state_dict())
# best checkpoint is saved automatically during training as cfg.best_weights_name when val improves

df = pd.DataFrame({"step": np.arange(len(losses)), "loss": losses})
save_csv(os.path.join(run_dir, "train_log.csv"), df)

meta = pack_config(params, _cfg_final, extra={"policy": policy, "flex_sss": flex.by_regime})
save_json(os.path.join(run_dir, "meta.json"), meta)

pointer_path = save_selected_run(_cfg_final.artifacts_root, policy, run_dir)
print("Saved to", run_dir)
print("Selected run pointer:", pointer_path)


In [ ]:
# Final cleanup (helps when running multiple models sequentially)
import gc
try:
    del trainer
except NameError:
    pass
try:
    del net
except NameError:
    pass
_ = gc.collect()
print('final gc.collect() done')
